In [1]:
import pickle
import pandas as pd

In [2]:
df = pd.read_csv("data/heart.csv")
df.head(2)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1


Abrimos todos los *transformers* que guardamos cuando hicimos el modelo

In [3]:
# encoding ChestPainType
with open('data/encodingChestPainType.pkl', 'rb') as chest:
    encoding_chestpain = pickle.load(chest)
    
# encoding Angina
with open('data/encodingExerciseAngina.pkl', 'rb') as angina:
    encoding_angina = pickle.load(angina)

    
# encoding Fasting
with open('data/encodingFastingBS.pkl', 'rb') as fasting:
    encoding_fasting = pickle.load(fasting)

# encoding Resting
with open('data/encodingRestingECG.pkl', 'rb') as resting:
    encoding_resting = pickle.load(resting)

# encoding Slope
with open('data/encodingST_Slope.pkl', 'rb') as slope:
    encoding_slope = pickle.load(slope)

# encoding Sex
with open('data/encodingSex.pkl', 'rb') as sex:
    encoding_sex = pickle.load(sex)

# estandarización
with open('data/estandarizacion.pkl', 'rb') as estandarizacion:
    estandarizacion = pickle.load(estandarizacion)


In [4]:
# cargamos el mejor modelo
with open('data/mejor_modelo.pkl', 'rb') as modelo:
    modelo = pickle.load(modelo)

Nos preparamos para hacer la predicción

In [5]:
# recordamos como eran los datos que nos pasaban de origen
df.sample(4)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
104,46,M,ASY,118,186,0,Normal,124,N,0.0,Flat,1
13,49,M,ASY,140,234,0,Normal,140,Y,1.0,Flat,1
625,59,M,ASY,135,234,0,Normal,161,N,0.5,Flat,0
672,60,F,NAP,120,178,1,Normal,96,N,0.0,Up,0


In [6]:
# le pedimos al médico que nos de los datos del paciente
resultados = {"Age" : int(input("¿Cuál es tu edad?")),
"Sex" : input("¿Cuál es tu genero?"),
"ChestPainType" : input("¿Tuviste dolor de pecho?"),
"RestingBP" : int(input("¿Cual es tu presion arterial?")),
"Cholesterol" : int(input("¿Cual es tu colesterol?")),
"FastingBS" : int(input("¿Cual es tu nivel de azucar?")),
"RestingECG" : input("¿Cual es tu resultado del electrocardiograma?"),
"MaxHR" : int(input("¿Cual es tu frecuencia cardiaca?")),
"ExerciseAngina" : input("¿Angina inducida?"),
"Oldpeak" : float(input("oldpeak?")),
"ST_Slope" : input("¿Cómo era la pendiente?")}


In [7]:
# convertimos los datos del paciente a dataframe para poder trabajar con ellos. 
df_resultados = pd.DataFrame(resultados, index=[0])
df_resultados

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope
0,28,F,ASY,120,50,0,Normal,160,N,1.5,Flat


In [8]:
# estandarizamos todas las columnas que son de tipo numérico usando el transformer que nos guardamos cuando hicimos el modelo

estandarizadas = pd.DataFrame(estandarizacion.transform(df_resultados[["Age", "RestingBP", "Cholesterol", 
                           "MaxHR", "Oldpeak"]]), columns = ["Age", "RestingBP", "Cholesterol", 
                           "MaxHR", "Oldpeak"])
estandarizadas

,Age,RestingBP,Cholesterol,MaxHR,Oldpeak
0,-2.706015,-0.71074,-4.047141,0.91358,0.700363


In [9]:
df_resultados.head(1)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope
0,28,F,ASY,120,50,0,Normal,160,N,1.5,Flat


In [10]:
# hacemos el encoding del resto de las variables

chest = pd.DataFrame(encoding_chestpain.transform(df_resultados[["ChestPainType"]]), columns= ["ChestPainType"])
angina = pd.DataFrame(encoding_angina.transform(df_resultados[["ExerciseAngina"]]), columns= ["ExerciseAngina"])
resting = pd.DataFrame(encoding_resting.transform(df_resultados[["RestingECG"]]), columns= ["RestingECG"])
slope = pd.DataFrame(encoding_slope.transform(df_resultados[["ST_Slope"]]), columns= ["ST_Slope"])
fasting = pd.DataFrame(encoding_fasting.transform(df_resultados[["FastingBS"]]), columns= ["FastingBS"])
sex = pd.DataFrame(encoding_sex.transform(df_resultados[["Sex"]]).toarray(), columns=["Sex_F", "Sex_M"])


In [12]:
sex

,Sex_F,Sex_M
0,1.0,0.0


In [11]:
# unimos todos los conjuntos de datos que hemos ido generando con los transformers
df_final = pd.concat([estandarizadas, sex, chest, angina, resting, slope, fasting], axis = 1)
df_final

,Age,RestingBP,Cholesterol,MaxHR,Oldpeak,Sex_F,Sex_M,ChestPainType,ExerciseAngina,RestingECG,ST_Slope,FastingBS
0,-2.706015,-0.71074,-4.047141,0.91358,0.700363,1.0,0.0,0,0,0,0,0


In [13]:
# cambiamos el orden de las columnas ya que estas deben estar en el mismo orden de como entrenamos al modelo
df_final = df_final.reindex(['Age', 'ChestPainType', 'RestingBP', 'Cholesterol', 'FastingBS',
       'RestingECG', 'MaxHR', 'ExerciseAngina', 'Oldpeak', 'ST_Slope',
       'Sex_F', 'Sex_M'], axis=1)

In [14]:
# hacemos la predicción para nuestro nuevo paciente
modelo.predict(df_final)

array([0])

In [15]:
# ¿Cuál es la probabilidad de que le un infarto al paciente?

modelo.predict_proba(df_final)

array([[0.55818977, 0.44181023]])

[0, 1]